In [14]:
# %pip install tabulate

In [15]:
import numpy as np
import pandas as pd
from random import randrange
import csv

pd.set_option("display.precision", 8)

# Agglomerative clustering

## Class

In [16]:
class AgglomerativeClustering:
    
    def __init__(self,n_clusters=2,linkage="single"):
        
        self.n_clusters = n_clusters
        self.linkage = linkage

    def fit_predict(self,X):
        '''fitting X and predicting labels'''

        n=X.shape[0] #number of datapoints     
        d=self.d_matrix(X) #generating distance matrix
        cluster=self.get_initial_cluster(n) #assigning initial clusters to datapoints

        s=set(range(n))     
        for _ in range(n-self.n_clusters): 
            p,q=np.unravel_index(np.argmin(d, axis=None), d.shape)
            t_set=s-{p,q}
            d=self.update_d(d,p,q,t_set,self.linkage) #updating distance matrix
            cluster=self.update_cluster(cluster,p,q) #updating clusters
            s=s-{max(p,q)} #removing utilised datapoint

        decor_l=[]
        for v in cluster.values():
            decor_l.append(v)
        
        self.labels_= self.clustertolabels(decor_l)
        return self.labels_

    def clustertolabels(self,clusters):
        '''formatting clusters to labels'''
        
        ln = sum([len(c) for c in clusters])
        labels = np.zeros(ln,dtype = int)
        ind = -1
        for c in clusters:
            ind+=1
            for i in c:
                labels[i] = ind
        return labels


    def d_matrix(self,data):
        '''upper triangular distance matrix'''

        n=data.shape[0]  
        d=np.empty(shape=[n,n]) 
        d.fill(np.inf)  
        for i in range(n-1):
            for j in range(i+1,n):
                d[i,j]=distance(data[i],data[j]) 
        return d

    
    def get_initial_cluster(self,n):
        '''function to define initial clusters'''
        
        c={}
        for i in range(n):
            c[i]={i}   
        return c

   
    def update_d(self,d,p,q,t_set,linkage):
        '''updating the distance matrix with distances according to the linkage criteria'''

        for i in t_set: 
            
            u,v=min(i,p),max(i,p) 
            w,x=min(i,q),max(i,q)
            if(linkage=="complete"):
                t=max(d[u,v],d[w,x])
            elif(linkage=="average"):
                t=(d[u,v]+d[w,x])/2
            else:     
                t=min(d[u,v],d[w,x])
        
            d[u,v]=t
            d[w,x]=t
            
        m_pq=max(p,q)
        d[m_pq,:]=np.inf
        d[:,m_pq]=np.inf
        return d


    def update_cluster(self,c,p,q):
        '''updating centroids with merging'''

        i=c.pop(max(p,q))
        m=min(p,q)
        c[m]=c[m].union(i) 
        return c

## Distance Matrix for Agglomerative Clustering

In [17]:
def distance(pt1,pt2):
    '''Calculate distance between two points'''
    
    dim=len(pt1)  
    s=0
    for i in range(dim):
        s+=(pt1[i]-pt2[i])**2 
    dist=np.sqrt(s)  
    return dist

# KNN

## Main KNN functions

In [18]:
def e_dist(sample, inputs):
    '''function to calculate euclidian distance'''
    
    return np.power(np.sum(np.power(sample - inputs, 2), axis=1), 0.5)

def lab_csfy(k, sorted_labels):
    '''function to determine the label by classifying'''

    kns = sorted_labels[:k] #get first k labels sorted based on increasing distances
    
    #getting the most frequent out of k labels
    uniq_lab = np.unique(kns)
    count = []
    for i in uniq_lab:
        cnt = np.count_nonzero(kns == i)
        count.append(cnt)

    return uniq_lab[np.argmax(count)]

def knn_main(sample, k, X, y):
    '''main classification function'''
  
    labels = list(y)
    inputs = list(X)

    e_d = e_dist(sample, inputs) #getting euclidian distance from every datapoint

    labeled_euclidian = np.vstack((e_d, labels)) #mapping distances and associated labels in 2D array

    sorted_euclidian = labeled_euclidian.T[labeled_euclidian.T[:, 0].argsort()] #sort the above array based on increasing distances
    sorted_labels = sorted_euclidian.T[1] #segregating labels for classifying

    return lab_csfy(k, sorted_labels)

## Creating new features for the train and test data

### Splitting data in train and test sets and returning final sets

In [19]:
def datasets_for_KNN(df, n, linkage): 
  '''
    assigning cluster ids as new species(labels); updating and returning training and testing data for subsequent KNN classifier
    df -> input dataframe
    n -> no.of clusters
  '''
  
  df = df.iloc[:,:-1].reindex(np.random.permutation(df.index)).reset_index(drop = True) #shuffling the rows of dataframe

  #splitting the dataframe into train and test based on given fraction
  training_data=df.sample(frac=0.7,random_state=1000).reset_index(drop = True)
  testing_data=df.drop(training_data.index).sample(frac=1.0).reset_index(drop = True)

  #generating cluster ids for, and appending them to training data 
  clustering = AgglomerativeClustering(n_clusters=n,linkage=linkage)
  pred_clusters = clustering.fit_predict(training_data.values)
  training_data['clst_ids'] = pred_clusters
  
  return (train_test_generation(n, training_data, testing_data))

### Generating new features pertaining to the association of a train set datapoint with each cluster

In [20]:
def train_test_generation(n, training_data, testing_data):
  '''
    generating new features based on "cluster centroids" as cluster representatives
    n -> no. of clusters
    training_data -> training_data
    testing_data -> testing_data
  '''

  for i in range(n):
    indexes = np.where(training_data['clst_ids'] == i) #getting the indexes of the rows where cluster id = i
    df_i = training_data.iloc[indexes[0], :-1].reset_index(drop = True) #creating intermediate dataframe of corresponding cluster id
    centroid = list(df_i.mean()) #getting the centroid of the corresponding cluster id of training data
    training_features = training_data.iloc[:,:-1]
    col = "feature_" + str(i)
    
    tr_list = []
    #appending the new features to the training data
    for point in training_features.values:
      tr_list.append(np.power(np.sum(np.square(point-centroid)), 0.5))
    training_data[col] = tr_list
    training_data[col] = scale_column(training_data, col)
    training_data['clst_ids'] = training_data.pop('clst_ids')

    te_list = []
    #appending the new features to the testing data calculated based on the centroid of training data
    for point in testing_data.values:
      te_list.append(np.power(np.sum(np.square(point-centroid)), 0.5))
    testing_data[col] = te_list
    testing_data[col] = scale_column(testing_data, col)

  return training_data, testing_data

### Scaling the generated features w.r.t mean and std deviation

In [21]:
def scale_column(df, col):
  mean = df[col].mean()
  std = df[col].std()
  return((df[col] - mean)/(mean - std))

# K-Fold Cross Validation

## Main Functions

In [22]:
def cv_sublists(dataset, folds):
    '''Function to create folds of the data'''

    nested_list = [] #nested list of "folds" no. of sublists 
    df_cv = dataset
    fold_size = int(df_cv.shape[0] / folds) #length of each sublist
        
    for i in range(folds): #loop to append each fold as a sublist
        fold = []
        
        while len(fold) < fold_size: #loop to add items to the sublist
            
            #selecting and appending a random row to sublist
            r = randrange(0, df_cv.shape[0]) 
            index = df_cv.index[r]
            fold.append(df_cv.loc[index].values.tolist())
            
            df_cv = df_cv.drop(index) #deleting the row added to the sublist
    
        nested_list.append(np.asarray(fold))
            
    return nested_list 

## Function for calculating accuracy

In [23]:
def compute_accuracy(original, predicted):
    count = 0
    for i in range(len(original)):
        if original[i] == predicted[i]:
            count += 1
    return count / float(len(original))

## Function to implement K-fold cross validation and return the results 

In [24]:
def kfcv_results(dataset, f, k):
    '''cv -> intermediate training set of concatenated n-1 folds'''
    nested_list=cv_sublists(dataset,f)
    result=[]
    # determine training data of f-1 sublists and testing data of 1 sublist 
    for i in range(f):
        r = list(range(f))
        r.pop(i)
        
        for j in r :
            if j == r[0]:
                cv_train = nested_list[j]
            else:    
                cv_train = np.concatenate((cv_train,nested_list[j]), axis=0)
        
        predictions = []
        for sample in nested_list[i][:,:-1]:
            prediction = knn_main(sample, k, cv_train[:,:-1], cv_train[:,-1])
            predictions.append(prediction)
            
        acc = compute_accuracy(nested_list[i][:,-1], predictions)   
        result.append(acc) 
        
    return result

# Executing on the Seed Dataset

In [25]:
df = pd.read_csv("log.csv", header = None)

In [26]:
clusters = [3,4,5,6]
neighbors = [3,4,5,6]
Accuracy_linkage = []
kfolds = 7
    
for linkage in ['single', 'complete', 'average']:
    Accuracy = []
    results_df = pd.DataFrame()
    for n in clusters:
        
        training_data, testing_data = datasets_for_KNN(df, n, linkage) #generating training and testing data inclusive of new features

        knn_acc = []
        for k in neighbors:
            res = kfcv_results(training_data, kfolds, k)
            acc = sum(res)/len(res) #computing accuracy of a particular value of k by averaging accuracies in each k-folds iteration
            knn_acc.append(acc)
        Accuracy.append(sum(knn_acc)/len(knn_acc)) #computing accuracy of cluster by averaging accuracies of different number of neighbors

    results_df['No of Clusters'] = clusters
    results_df['Accuracy'] = Accuracy

    print(f"\033[1m{linkage} Linkage\033[0m\n")
    print(results_df.to_markdown(index=False)) 
    print(f"\nOptimal Clusters: {results_df['No of Clusters'][results_df['Accuracy'].idxmax()]}, Accuracy: {results_df['Accuracy'].max()}\n\n")

single Linkage

|   No of Clusters |   Accuracy |
|-----------------:|-----------:|
|                3 |   0.989796 |
|                4 |   0.984694 |
|                5 |   0.957483 |
|                6 |   0.965986 |

Optimal Clusters: 3, Accuracy: 0.989795918367347


complete Linkage

|   No of Clusters |   Accuracy |
|-----------------:|-----------:|
|                3 |   0.965986 |
|                4 |   0.981293 |
|                5 |   0.95068  |
|                6 |   0.955782 |

Optimal Clusters: 4, Accuracy: 0.9812925170068028


average Linkage

|   No of Clusters |   Accuracy |
|-----------------:|-----------:|
|                3 |   0.97449  |
|                4 |   0.981293 |
|                5 |   0.947279 |
|                6 |   0.957483 |

Optimal Clusters: 4, Accuracy: 0.9812925170068028




# Predicting on Testing data

In [27]:
training_data, testing_data = datasets_for_KNN(df, 3, 'average')

In [28]:
training_data.head()

,0,1,2,3,4,5,6,feature_0,feature_1,feature_2,clst_ids
0,17.12,15.55,0.8892,5.850,3.566,2.858,5.746,-1.63002949,1.34452798,-0.45591203,0
1,18.17,16.26,0.8637,6.271,3.512,2.853,6.273,-1.85112793,2.25203607,0.12187518,0
2,12.10,13.15,0.8793,5.105,2.941,2.201,5.056,1.17998568,-1.38372805,-0.17649579,1
3,20.71,17.23,0.8763,6.579,3.814,4.451,6.451,-0.63014014,3.76546103,1.34469853,0
4,13.02,13.76,0.8641,5.395,3.026,3.373,4.825,0.59276997,-2.04756887,-0.28552531,1


In [29]:
testing_data.head()

,0,1,2,3,4,5,6,feature_0,feature_1,feature_2
0,15.26,14.85,0.8696,5.714,3.242,4.543,5.314,-0.52597800,-0.38827005,-0.53229249
1,14.49,14.61,0.8538,5.715,3.113,4.116,5.396,-0.21336248,-0.96475643,-0.62323201
2,11.56,13.31,0.8198,5.363,2.683,4.062,5.182,1.33893833,-2.06448273,0.38386626
3,13.94,14.17,0.8728,5.585,3.150,2.124,5.012,0.19765383,-0.88159793,-1.08867838
4,11.18,13.04,0.8266,5.220,2.693,3.332,5.001,1.55727580,-1.59124524,0.36397377


In [30]:
predictions = []
for sample in testing_data.values:
  prediction = knn_main(sample, 4, training_data.iloc[:,:-1].values, training_data.iloc[:,-1].values)
  predictions.append(prediction)

In [31]:
print(predictions[:21])
print(predictions[21:42])
print(predictions[42:])

[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 2.0, 2.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[2.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]


# KNN on Original Seed  Dataset

In [32]:
import pandas as pd
df = pd.read_csv("log.csv", header=None)

In [33]:
df = df.sample(frac=1).reset_index(drop = True)
df_train = df.iloc[:147, :].reset_index(drop=True)
df_test = df.iloc[147:, :].reset_index(drop=True)

In [34]:
X = df_train.iloc[:, :-1].values
y = df_train.iloc[:, -1].values
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values

In [35]:
Accuracy = []
knn_df = pd.DataFrame()
for k in neighbors:
  res = kfcv_results(df_train.iloc[:147,:], 7, k)
  acc = sum(res)/len(res)
  # print(f"k = {k}, Accuracy = {acc}")
  Accuracy.append(acc)

knn_df['k'] = neighbors
knn_df['Accuracy'] = Accuracy
print(knn_df.to_markdown(index=False))
print("\n")
print("Average Accuracy:", sum(Accuracy)/len(Accuracy))

|   k |   Accuracy |
|----:|-----------:|
|   3 |   0.877551 |
|   4 |   0.897959 |
|   5 |   0.884354 |
|   6 |   0.877551 |


Average Accuracy: 0.8843537414965987
